In [20]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path
from datetime import datetime, timezone, timedelta

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import librosa

from sklearn.metrics import roc_auc_score, average_precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import json
import timm

from importlib import reload

logging.basicConfig(level=logging.ERROR)

from module import preprocess_lib, datasets_lib, utils_lib, models_lib, learning_lib, config_lib
reload(config_lib)

<module 'module.config_lib' from '/root/program/birdclef-2025/scripts/module/config_lib.py'>

In [21]:


class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = '/kaggle/input/birdclef-2025-0330'
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.RAW_DIR = '../data/raw/'
            self.PROCESSED_DIR = '../data/processed/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.models_dir = "../models/" # 全modelの保存先
            self.model_path = self.models_dir # 各モデルの保存先．学習時に動的に変更．
            
            self.spectrogram_npy = '../data/processed/baseline/birdclef2025_melspec_5sec_256_256.npy'
            
            self.pseudo_label_csv = "../data/result/pseudo_labels_baseline_7sec.csv"
            self.pseudo_melspec_npy = "../data/processed/train_soundscapes_0407/train_soundscapes_melspecs.npy"

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5.0 # 推論時のウィンドウサイズ
        self.TARGET_DURATION = 5.0 # データセット作成時のウィンドウサイズ
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000        

        # ===== Training Mode =====
        if mode == "train":
            self.seed = 42
            self.apex = False
            self.print_freq = 100
            self.num_workers = 2

            self.LOAD_DATA = True
            self.epochs = 10
            self.batch_size = 32
            self.criterion = 'BCEWithLogitsLoss'

            self.n_fold = 5
            self.selected_folds = [0, 1, 2, 3, 4]

            self.optimizer = 'AdamW'
            self.lr = 5e-4
            self.weight_decay = 1e-5
            self.scheduler = 'CosineAnnealingLR'
            self.min_lr = 1e-6
            self.T_max = self.epochs

            self.aug_prob = 0.5
            self.mixup_alpha_real = 0.5
            self.mixup_alpha_pseudo = 0.5
            
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            
            self.use_pseudo_mixup = False  # pseudo lableでmixupするかどうか
            self.pseudo_mix_prob = 0.4  # mixupでpseudo lableを使う確率
            self.pseudo_conf_threshold = 0.5
            

            if self.debug:
                self.epochs = 2
                self.selected_folds = [0]
                self.batch_size = 4
                

In [29]:
# debug trueにするとvalidationの数が1000に固定される．
cfg = CFG(mode="train", kaggle_notebook=False, debug=False)

In [30]:
utils_lib.set_seed(cfg.seed)

In [31]:
class BirdCLEFValidator:
    def __init__(self, cfg, df, datasets_lib, models_lib):
        self.cfg = cfg
        self.df = df
        self.datasets_lib = datasets_lib
        self.models_lib = models_lib
        self.label2index = {}
        self.index2label = {}
        self.spectrograms = None

        self._load_taxonomy()
        self._load_spectrograms()

    def _load_taxonomy(self):
        taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        species_ids = taxonomy_df['primary_label'].tolist()
        self.cfg.num_classes = len(species_ids)
        self.index2label = {i: label for i, label in enumerate(species_ids)}
        self.label2index = {label: i for i, label in enumerate(species_ids)}

    def _load_spectrograms(self):
        print(f"Loading pre-computed mel spectrograms from: {self.cfg.spectrogram_npy}")
        self.spectrograms = np.load(self.cfg.spectrogram_npy, allow_pickle=True).item()
        print(f"Loaded {len(self.spectrograms)} spectrograms")

    def _get_val_df(self, fold):
        skf = StratifiedKFold(n_splits=self.cfg.n_fold, shuffle=True, random_state=self.cfg.seed)
        _, val_idx = list(skf.split(self.df, self.df['primary_label']))[fold]
        return self.df.iloc[val_idx].reset_index(drop=True)

    def _get_val_loader(self, val_df):
        val_dataset = self.datasets_lib.BirdCLEFDatasetFromNPY(val_df, self.cfg, self.spectrograms, mode='valid')
        return DataLoader(val_dataset, batch_size=self.cfg.batch_size, shuffle=False,
                          num_workers=self.cfg.num_workers, pin_memory=True,
                          collate_fn=self.datasets_lib.collate_fn)

    def _load_model(self, model_path):
        model = self.models_lib.BirdCLEFModelForTrain(self.cfg).to(self.cfg.device)
        state = torch.load(model_path, map_location=self.cfg.device)
        model.load_state_dict(state['model_state_dict'])
        return model.eval()

    def _predict(self, model, loader):
        model.eval()
        all_outputs, all_targets, all_filenames = [], [], []
        with torch.no_grad():
            for batch in tqdm(loader, desc="Validation"):
                if isinstance(batch['melspec'], list):
                    for melspec, target, filename in zip(batch['melspec'], batch['target'], batch['filename']):
                        inputs = melspec.unsqueeze(0).to(self.cfg.device)
                        output = model(inputs)
                        output = output[0] if isinstance(output, tuple) else output
                        all_outputs.append(output.detach().cpu().numpy())
                        all_targets.append(target.numpy())
                        all_filenames.append(filename)
                else:
                    inputs = batch['melspec'].to(self.cfg.device)
                    outputs = model(inputs)
                    outputs = outputs[0] if isinstance(outputs, tuple) else outputs
                    outputs = outputs.detach().cpu().numpy()
                    targets = batch['target'].numpy()
                    all_outputs.extend(outputs)
                    all_targets.extend(targets)
                    all_filenames.extend(batch['filename'])

        return np.array(all_outputs), np.array(all_targets), all_filenames

    def _calculate_auc(self, targets, outputs):
        probs = 1 / (1 + np.exp(-outputs))
        targets_bin = (targets >= 0.5).astype(int)
        aucs = [roc_auc_score(targets_bin[:, i], probs[:, i])
                for i in range(targets.shape[1]) if np.sum(targets_bin[:, i]) > 0]
        return np.mean(aucs) if aucs else 0.0

    def _calculate_map(self, targets, outputs):
        probs = 1 / (1 + np.exp(-outputs))
        targets_bin = (targets >= 0.5).astype(int)
        aps = [average_precision_score(targets_bin[:, i], probs[:, i])
               for i in range(targets.shape[1]) if np.sum(targets_bin[:, i]) > 0]
        return np.mean(aps) if aps else 0.0

    def evaluate_model_dir(self, model_dir):
        full_dir = os.path.join(self.cfg.models_dir, model_dir)
        print(f"\n🔍 Evaluating model directory: {full_dir}")

        fold_preds = {}
        fold_targets = {}

        for fold in range(self.cfg.n_fold):
            model_path = os.path.join(full_dir, f"model_fold{fold}.pth")
            if not os.path.exists(model_path):
                print(f"⛔️ model_fold{fold}.pth not found in {model_dir}")
                continue

            val_df = self._get_val_df(fold)
            val_loader = self._get_val_loader(val_df)
            self.val_loader = val_loader

            model = self._load_model(model_path)
            outputs, targets, filenames = self._predict(model, val_loader)

            fold_preds[fold] = outputs
            fold_targets[fold] = targets

            class_names = [self.index2label[i] for i in range(outputs.shape[1])]
            probs = 1 / (1 + np.exp(-outputs))
            df_preds = pd.DataFrame(probs, columns=class_names)
            df_preds.insert(0, "row_id", filenames)
            df_preds.to_csv(os.path.join(full_dir, f"predictions_fold{fold}.csv"), index=False)

        print(f"\n✅ Finished evaluation for model_dir: {model_dir}")


In [32]:
if __name__ == "__main__":
    print("\nLoading training data...")
    train_df = pd.read_csv(cfg.train_csv)

    model_dirs = [
    "baseline_fold0_7sec",
    ]
    validator = BirdCLEFValidator(cfg, train_df, datasets_lib, models_lib)
    # ====== 実行 ======
    for model_dir in model_dirs:
        validator.evaluate_model_dir(model_dir)



Loading training data...
Loading pre-computed mel spectrograms from: ../data/processed/baseline/birdclef2025_melspec_5sec_256_256.npy
Loaded 28564 spectrograms

🔍 Evaluating model directory: ../models/baseline_fold0_7sec
Found 5713 matching spectrograms for valid dataset out of 5713 samples


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Validation:   0%|          | 0/179 [00:00<?, ?it/s]

⛔️ model_fold1.pth not found in baseline_fold0_7sec
⛔️ model_fold2.pth not found in baseline_fold0_7sec
⛔️ model_fold3.pth not found in baseline_fold0_7sec
⛔️ model_fold4.pth not found in baseline_fold0_7sec

✅ Finished evaluation for model_dir: baseline_fold0_7sec


In [33]:
predictions = pd.read_csv(os.path.join(cfg.models_dir, "baseline_fold0_7sec/predictions_fold0.csv"))
predictions

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,1139490/CSA36389.ogg,3.054938e-03,1.058280e-01,2.319143e-03,1.007705e-03,2.178936e-03,2.727929e-05,4.128603e-04,2.188327e-02,5.162641e-01,...,0.000406,3.534887e-03,6.474329e-05,4.313095e-05,1.847345e-04,0.001277,2.272246e-04,0.000163,0.000101,0.000032
1,1192948/CSA36373.ogg,2.471087e-03,2.840580e-02,1.252309e-03,7.804363e-04,1.629637e-03,8.397186e-05,4.191425e-04,9.933868e-03,8.322847e-02,...,0.000661,4.886495e-03,6.982028e-05,2.876412e-04,2.224172e-04,0.000970,1.469607e-03,0.001522,0.000521,0.000265
2,126247/iNat1109254.ogg,3.416209e-05,1.367200e-04,2.759284e-05,3.600779e-03,2.902223e-04,3.520263e-05,6.545848e-06,5.316449e-05,5.131463e-05,...,0.000012,1.180046e-04,8.476275e-04,3.655616e-05,3.243734e-04,0.000118,3.845574e-05,0.000013,0.000243,0.000784
3,1346504/CSA18792.ogg,3.145532e-05,3.263174e-05,8.248750e-03,1.736439e-05,5.550703e-02,4.366834e-04,1.762758e-02,4.786416e-05,3.819636e-05,...,0.000092,6.286265e-04,2.839635e-04,3.721417e-05,1.007722e-05,0.000167,1.760580e-06,0.008490,0.000002,0.000002
4,134933/XC941298.ogg,4.412470e-04,8.149637e-04,5.512392e-04,1.767635e-04,1.359499e-03,1.168304e-02,5.199848e-03,4.320598e-04,6.110319e-04,...,0.008908,9.562799e-03,3.066562e-04,1.013359e-03,8.274140e-05,0.010744,2.752460e-04,0.033313,0.000604,0.000686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5708,ywcpar/iNat1169084.ogg,8.728448e-08,1.809688e-08,2.879015e-07,1.988114e-07,2.153236e-07,5.719475e-07,2.065921e-07,7.661289e-08,2.363093e-09,...,0.000082,2.915606e-06,4.493673e-06,9.403532e-07,1.810797e-08,0.000216,9.397358e-08,0.000110,0.000433,0.970162
5709,ywcpar/iNat136657.ogg,3.489906e-07,1.144913e-07,9.647761e-07,4.845985e-07,1.697961e-06,2.333507e-06,1.527326e-05,3.542739e-07,1.032100e-08,...,0.000079,1.346929e-06,3.678536e-06,2.684272e-07,3.510388e-07,0.003499,4.291385e-07,0.000081,0.000061,0.143911
5710,ywcpar/iNat33510.ogg,7.274562e-06,4.208633e-05,2.806115e-06,6.700125e-06,2.391455e-06,4.106619e-06,2.612631e-06,1.676767e-05,1.306048e-05,...,0.000298,1.000403e-04,8.141595e-06,1.938280e-06,2.561105e-05,0.001789,6.363018e-04,0.000419,0.000565,0.075480
5711,ywcpar/iNat370914.ogg,8.195959e-08,3.531020e-08,1.046206e-07,6.872435e-08,3.120099e-08,6.567136e-08,4.288561e-08,7.515929e-08,2.842746e-09,...,0.000059,2.196716e-07,1.745988e-07,4.166304e-07,7.619679e-09,0.000145,8.069190e-06,0.000654,0.002307,0.884393


In [28]:
filenames

['1139490/CSA36389.ogg',
 '1192948/CSA36373.ogg',
 '126247/iNat1109254.ogg',
 '1346504/CSA18792.ogg',
 '134933/XC941298.ogg',
 '135045/iNat1207347.ogg',
 '135045/iNat327127.ogg',
 '1462711/CSA36371.ogg',
 '1462737/CSA36386.ogg',
 '1564122/CSA34195.ogg',
 '21038/iNat297879.ogg',
 '21211/XC882647.ogg',
 '21211/XC882650.ogg',
 '21211/XC882655.ogg',
 '21211/XC896832.ogg',
 '21211/XC896863.ogg',
 '21211/XC913881.ogg',
 '21211/XC913893.ogg',
 '21211/XC913998.ogg',
 '21211/XC917471.ogg',
 '21211/XC920881.ogg',
 '21211/XC925906.ogg',
 '21211/XC925925.ogg',
 '21211/XC928710.ogg',
 '21211/iNat359445.ogg',
 '21211/iNat361222.ogg',
 '22333/XC890507.ogg',
 '22333/XC894982.ogg',
 '22333/iNat1237907.ogg',
 '22333/iNat144601.ogg',
 '22333/iNat53679.ogg',
 '22333/iNat54095.ogg',
 '22333/iNat585061.ogg',
 '22333/iNat758022.ogg',
 '22333/iNat815140.ogg',
 '22333/iNat983570.ogg',
 '22973/XC882794.ogg',
 '22973/iNat318716.ogg',
 '22973/iNat326966.ogg',
 '22973/iNat327135.ogg',
 '22973/iNat343840.ogg',
 '22